# 225. Implement Stack using Queues

[Link to Problem](https://leetcode.com/problems/implement-stack-using-queues/)

### Description
Implement a last-in-first-out (LIFO) stack using only two queues. The implemented stack should support all the functions of a normal stack (`push`, `top`, `pop`, and `empty`).

Implement the `MyStack` class:

- `void push(int x)` Pushes element x to the top of the stack.
- `int pop()` Removes the element on the top of the stack and returns it.
- `int top()` Returns the element on the top of the stack.
- `boolean empty()` Returns `true` if the stack is empty, `false` otherwise.

---
**Notes:**
- You must use **only** standard operations of a queue, which means only `push to back`, `peek/pop from front`, `size`, and `is empty` operations are valid.

---
**Example 1:**

Input:
`["MyStack", "push", "push", "top", "pop", "empty"]`
`[[], [1], [2], [], [], []]`

Output:
`[null, null, null, 2, 2, false]`

Explanation:
MyStack myStack = new MyStack();
myStack.push(1);
myStack.push(2);
myStack.top();   // return 2
myStack.pop();   // return 2
myStack.empty(); // return False

---
**Constraints:**
- `1 <= x <= 9`
- At most `100` calls will be made to `push`, `pop`, `top`, and `empty`.
- All the calls to `pop` and `top` are valid.


In [ ]:
from collections import deque

class MyStack:

    def __init__(self):
        self.Q = deque([])
        self.T = deque([])

    def push(self, x: int) -> None:
        self.Q.appendleft(x)

    def pop(self) -> int:
        while len(self.Q) > 1:
            self.T.appendleft(
                self.Q.pop()
            )
        result = self.Q.pop()
        while len(self.T) > 0:
            self.Q.appendleft(
                self.T.pop()
            )
        return result

    def top(self) -> int:
        while len(self.Q) > 1:
            self.T.appendleft(
                self.Q.pop()
            )
        result = self.Q.pop()
        self.T.appendleft(result)
        while self.T:
            self.Q.appendleft(
                self.T.pop()
            )
        return result

    def empty(self) -> bool:
        return len(self.Q) == 0

Excellent — let’s do a **code review** of your solution for
🧱 **LeetCode 225. Implement Stack using Queues**, focusing on **algorithmic quality, correctness, and interview readability**.

---

## ✅ Strengths

### 1. **Functionality**

Your solution correctly:

* Implements all 4 required operations (`push`, `pop`, `top`, `empty`).
* Uses **only queue operations**: `appendleft`, `pop`, and `len()` (valid for deque-based queue behavior).
* Maintains correct **LIFO order** using two queues.

### 2. **Correctness**

* Works correctly on standard test cases (e.g., `[1,2] → pop() returns 2`).
* `empty()` behaves correctly.
* `top()` restores state after peeking — that’s a good attention to detail.

### 3. **Conceptual Alignment**

* You clearly understand that stacks can be simulated via queues by **reordering** the elements.

---

## ⚠️ Issues & Improvement Areas

Let’s go step-by-step.

---

### 🧩 1. **Inefficient Push Strategy**

Right now, your `push()` simply does:

```python
self.Q.appendleft(x)
```

Then every time you `pop()` or `top()`, you shuffle elements back and forth between `Q` and `T`.

That means:

* `push()` is **O(1)** (cheap)
* `pop()` and `top()` are **O(n)** (expensive — full transfers)

✅ It works.
❌ But it’s not optimal for interviews — we can **invert the logic** to make it cleaner.

---

### 🧠 Alternative Approach (More Common & Interview-Favored)

Make `push()` do the reordering once, so `pop()` and `top()` become O(1):

```python
class MyStack:
    def __init__(self):
        self.q1 = deque()
        self.q2 = deque()

    def push(self, x: int) -> None:
        # Step 1: enqueue x into q2
        self.q2.append(x)
        # Step 2: push all elements from q1 to q2
        while self.q1:
            self.q2.append(self.q1.popleft())
        # Step 3: swap q1 and q2
        self.q1, self.q2 = self.q2, self.q1

    def pop(self) -> int:
        return self.q1.popleft()

    def top(self) -> int:
        return self.q1[0]

    def empty(self) -> bool:
        return not self.q1
```

* ✅ **Push:** O(n)
* ✅ **Pop / Top:** O(1)
* ✅ Simpler logic and avoids double transfer.

This version is more common in **interviews** because:

* It demonstrates understanding of **trade-offs between operations**.
* It’s concise and intuitive (stack top always front of `q1`).

---

### 🧮 2. **Complexity Analysis**

Your version:

* `push`: O(1)
* `pop` / `top`: O(n) (due to two full transfers between `Q` and `T`)
* `empty`: O(1)
* Space: O(n)

Alternative version:

* `push`: O(n)
* `pop`, `top`: O(1)
* Space: O(n)

👉 Both are valid — the difference is **where the cost is paid**.

---

### 🧾 3. **Readability**

You use `appendleft` + `pop`, but typical queue usage with `deque` uses:

* `append()` → enqueue
* `popleft()` → dequeue

Using `appendleft` / `pop` is valid but counterintuitive (flipped semantics).
If you’re asked to implement "using queue operations," it’s clearer to stick with the **front = left** convention:

```python
append() → enqueue to back  
popleft() → dequeue from front
```

---

### 🧠 4. **Floyd’s Cycle–Style Analogy (Optional Wow Factor)**

In this problem, we can draw an analogy like this:

> The two-queue transfer logic is like a **slow–fast pointer** system:
>
> * One queue (`Q`) is the “active” state.
> * The other (`T`) acts as a “shadow” buffer, temporarily reversing order.
>
> Each push/pop toggles between these two states — just like cycle detection flips between slow and fast pointers.
> You’re effectively *simulating a stack cycle* via controlled state transitions between queues.

This kind of mental analogy shows your ability to see *algorithmic patterns* even in data structure problems — interviewers love that.

---

### 🧩 5. **Possible Further Optimization**

If allowed to use **a single queue**, we can avoid maintaining two queues:

In [1]:
class MyStack:
    def __init__(self):
        self.q = deque()

    def push(self, x: int) -> None:
        self.q.append(x)
        for _ in range(len(self.q) - 1):
            self.q.append(self.q.popleft())  # rotate the queue

    def pop(self) -> int:
        return self.q.popleft()

    def top(self) -> int:
        return self.q[0]

    def empty(self) -> bool:
        return not self.q

* Still valid under “queue-only” rules.
* Only **one queue** needed.
* Very popular among LeetCode discussions and interviews.

---

## 🔍 Summary Table

| Approach                   | Queues Used | Push | Pop  | Top  | Empty | Space | Notes                         |
| -------------------------- | ----------- | ---- | ---- | ---- | ----- | ----- | ----------------------------- |
| **Your Version**           | 2           | O(1) | O(n) | O(n) | O(1)  | O(n)  | Functional but less efficient |
| **Optimized (push heavy)** | 2           | O(n) | O(1) | O(1) | O(1)  | O(n)  | Interview-preferred           |
| **One Queue Rotation**     | 1           | O(n) | O(1) | O(1) | O(1)  | O(n)  | Most elegant                  |

---

### ✅ Final Thoughts

* Your implementation is **correct** and **demonstrates clear logic**.
* For interviews:

  * Start with your version (shows core understanding).
  * Then mention the **optimized push-heavy** or **single-queue rotation** version.
  * Add a quick complexity trade-off comparison.
  * Optional: mention the **queue-as-state analogy** for extra depth.

---

Would you like me to show a **visual step-by-step trace** (queue contents changing during `push` and `pop`) to help you explain it clearly in an interview?
